In [1]:
!pip install ipywidgets jupyter transformers torch pandas bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.2 MB/s eta 0:00:00


# FAQ Generator Notebook
This notebook generates 20 accurate, topic-specific FAQs using the Phi-3-mini model. Features:
- Interactive UI with ipywidgets.
- Generates answers (optional).
- Exports to CSV.
- Debug mode for troubleshooting.
## Setup
Run this cell to install dependencies (if not already installed in your virtual environment).

In [10]:
!pip install ipywidgets jupyter transformers torch pandas bitsandbytes accelerate

## Imports and Model Loading

This section imports the required libraries and loads the Phi-3-mini model and its tokenizer.

In [13]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import re
import pandas as pd
import io
import torch

# Cache model loading
def load_model():
    try:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Using device: {device}")

        # Determine dtype based on device
        dtype = torch.bfloat16 if device == 'cuda' else torch.float32

        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained('microsoft/Phi-3-mini-4k-instruct')
        tokenizer.padding_side = 'left' # Set padding side for generation

        # Load model
        model = AutoModelForCausalLM.from_pretrained(
            'microsoft/Phi-3-mini-4k-instruct',
            dtype=dtype # Use dtype here
        )

        # Create pipeline
        generator = pipeline(
            'text-generation',
            model=model,
            tokenizer=tokenizer,
            device=device
        )
        return generator
    except Exception as e:
        print(f'Error loading model: {str(e)}')
        return None

generator = load_model()
if generator is None:
    raise Exception('Model failed to load. Check dependencies or internet.')

Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


## Interactive FAQ Generator

Use the interface below to generate FAQs for your desired topic.

In [15]:
# Widgets for UI
topic_input = widgets.Text(
    value='',
    placeholder='e.g., Electric cars',
    description='Topic:',
    layout={'width': '600px'}
)
max_tokens_slider = widgets.IntSlider(
    value=600,
    min=300,
    max=800,
    step=50,
    description='Max Tokens:',
    style={'description_width': 'initial'}
)
answers_check = widgets.Checkbox(
    value=False,
    description='Generate Answers (slower)',
    style={'description_width': 'initial'}
)
debug_check = widgets.Checkbox(
    value=False,
    description='Show Debug Output',
    style={'description_width': 'initial'}
)
generate_button = widgets.Button(
    description='Generate 20 FAQs',
    button_style='primary',
    tooltip='Click to generate',
    layout={'width': '200px'}
)
output = widgets.Output()

# Store results
faqs = []
answers = []

# Generate function
def on_generate_button_clicked(b):
    with output:
        clear_output()
        if not topic_input.value.strip():
            print('Please enter a topic!')
            return

        print('Generating accurate FAQs...')
        try:
            global faqs, answers
            faqs = []
            answers = []

            # Prompt for 20 FAQs
            prompt = f'<|user|>\nGenerate exactly 20 highly relevant and accurate frequently asked questions about "{topic_input.value}". Make them concise, diverse, and specific to the topic. Output ONLY a numbered list from 1 to 20, each on a new line. No introductions or extras.<|end|>\n<|assistant|>\n'

            result = generator(
                prompt,
                max_new_tokens=max_tokens_slider.value,
                do_sample=False,
                temperature=0.1,
                pad_token_id=generator.tokenizer.eos_token_id
            )

            generated = result[0]['generated_text']
            assistant_start = generated.find('<|assistant|>') + len('<|assistant|>\n')
            faqs_text = generated[assistant_start:].strip()

            # Parse
            faqs_raw = faqs_text.split('\n')
            for line in faqs_raw:
                line = line.strip()
                if re.match(r'^\d+\.\s*.+', line):
                    faqs.append(line)
                elif line and len(line) > 5:
                    faqs.append(f'{len(faqs) + 1}. {line}')

            # Ensure 20
            while len(faqs) < 20:
                faqs.append(f'{len(faqs) + 1}. Additional FAQ on {topic_input.value} (expand topic for more).')
            faqs = faqs[:20]

            # Answers
            if answers_check.value:
                print('Generating answers...')
                for faq in faqs:
                    ans_prompt = f'<|user|>\nProvide a short, accurate answer to: {faq}<|end|>\n<|assistant|>\n'
                    ans_result = generator(ans_prompt, max_new_tokens=100, do_sample=True, temperature=0.3)
                    ans_text = ans_result[0]['generated_text'].split('<|assistant|>')[-1].strip()
                    answers.append(ans_text)

            # Debug
            if debug_check.value:
                print('--- Debug: Generated Text ---')
                print(generated)
                print('--- Debug: Parsed FAQs ---')
                print(faqs)

            # Display FAQs
            print(f'\nTop 20 Accurate FAQs for "{topic_input.value}":')
            for i, faq in enumerate(faqs, 1):
                print(f'Q{i}: {faq}')
                if answers_check.value and i-1 < len(answers):
                    print(f'Answer: {answers[i-1]}')
                print('---')

            # CSV export
            print('\nExporting to CSV...')
            df = pd.DataFrame({'Question Number': list(range(1, 21)), 'Question': faqs})
            if answers_check.value and answers:
                # Ensure answers list is not empty and matches the number of FAQs before adding to DataFrame
                if answers and len(answers) >= len(faqs):
                     df['Answer'] = answers[:len(faqs)]
                elif answers:
                     # Handle cases where fewer answers were generated than FAQs
                     print(f"Warning: Generated {len(answers)} answers for {len(faqs)} FAQs. Some FAQs will not have answers in the CSV.")
                     df['Answer'] = answers + [''] * (len(faqs) - len(answers)) # Pad with empty strings

            csv_buffer = io.StringIO()
            df.to_csv(csv_buffer, index=False)
            # Sanitize topic_input.value for filename
            safe_topic = re.sub(r'[^\w\s-]', '', topic_input.value).replace(' ', '_')
            filename = f'phi_faqs_{safe_topic}.csv'
            with open(filename, 'w') as f:
                f.write(csv_buffer.getvalue())
            print(f'Saved to {filename}')

            print(f'Generated {len(faqs)} FAQs successfully!')
        except Exception as e:
            print(f'Error: {str(e)}')

# Connect button
generate_button.on_click(on_generate_button_clicked)

# Display UI
display(widgets.VBox([
    widgets.HTML('<h1 style="color: #1f77b4;">❓ Smart FAQ Generator</h1>'),
    widgets.HTML('<p style="color: #ff7f0e;">Enter a topic to generate 20 accurate FAQs!</p>'),
    topic_input,
    max_tokens_slider,
    answers_check,
    debug_check,
    generate_button,
    output
]))